# Activity 5

This activity is an application of the Newton-Raphson method for univariate root-finding.

## Imports

If you wish to import other packages and/or modules,
add a blank line after importing `scipy.optimize` in the following code cell,
followed by the comment line `# Custom imports`,
after which your additional imports should be disclosed.
For example, to use NumPy, the succeeding code cell should show

>> ```python
>> import math as mt
>> 
>> import scipy.optimize as spo
>> 
>> # Custom imports
>> import numpy as np
>> ```

In [1]:
import math as mt

import scipy.optimize as spo

# Custom imports
import numpy as np

## Scenario

Suppose you have a second-order circuit that can be reduced into
an equivalent resistance $R$
parallel-connected to
an equivalent inductance $L$
as well as to
an equivalent capacitance $C$
(see [diagram](./act-05.png)).
The said model is only valid over the interval $4 \leq t \leq 21$ seconds.

Determine:
- the minimum voltage across the equivalent-inductance element,
  and
- when the voltage across the equivalent-resistance element is a minimum,

for when:
- $C = 500$ millifarads,
- $R = \frac{2}{3}$ ohms,
  and
- $L = 1000$ millihenrys,

provided that 
$V = 10$ volts
and
$I = 2$ amperes
at the start of the applicable interval.

In [2]:
C = 500e-3       
R = 2/3          
L = 1000e-3      
V0 = 10.0         
I0 = 2.0       


## Modelling

Derive $f\!\left(t\right)$ whose root corresponds to when the voltage across the equivalent-resistance element is a minimum.
Then, derive an expression of $V\!\left(t\right)$ across the equivalent-inductance element.

Construct three Python functions:
- `f()`, which implements $f\!\left(t\right)$;
- `df()`, which implements the derivative of $f\!\left(t\right)$;
  and
- `v_L()`, which implements $V\!\left(t\right)$.

Each of the said functions should take a single positional argument `t`.

In the succeeding code cell,
replace the `pass` statement with your code.

In [3]:
alpha = 1.0 / (2.0 * R * C)     
omega0 = 1.0 / mt.sqrt(L * C)  
# initial derivative from KCL: C*v'(0) + v(0)/R + iL(0) = 0
vprime0 = -(V0 / R + I0) / C
is_overdamped = alpha > omega0
if is_overdamped:
    sqrt_term = mt.sqrt(alpha**2 - omega0**2)
    s1 = -alpha + sqrt_term
    s2 = -alpha - sqrt_term
    A = (vprime0 - s2 * V0) / (s1 - s2)
    B = V0 - A
else:
    omega_d = mt.sqrt(max(0.0, omega0**2 - alpha**2))
    A = V0
    B = (vprime0 + alpha * A) / omega_d

def f(t):
    if is_overdamped:
        return A * s1 * mt.exp(s1 * t) + B * s2 * mt.exp(s2 * t)
    else:
        exp_term = mt.exp(-alpha * t)
        return exp_term * (
            -alpha * (A * mt.cos(omega_d * t) + B * mt.sin(omega_d * t))
            - A * omega_d * mt.sin(omega_d * t)
            + B * omega_d * mt.cos(omega_d * t)
        )

def df(t):
    if is_overdamped:
        return A * (s1**2) * mt.exp(s1 * t) + B * (s2**2) * mt.exp(s2 * t)
    else:
        exp_term = mt.exp(-alpha * t)
        return exp_term * (
            (alpha**2 - omega_d**2) * (A * mt.cos(omega_d * t) + B * mt.sin(omega_d * t))
            + 2 * alpha * omega_d * (A * mt.sin(omega_d * t) - B * mt.cos(omega_d * t))
        )

def v_L(t):
    if is_overdamped:
        return A * mt.exp(s1 * t) + B * mt.exp(s2 * t)
    else:
        return mt.exp(-alpha * t) * (A * mt.cos(omega_d * t) + B * mt.sin(omega_d * t))

Use the Newton-Raphson method to find the root of $f\!\left(t\right)$.

Define the Python variable `T0` that will store the initial estimate of the root.
Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as convergence information,
and store the said outputs in Python variables `t_nr` and `t_nr_info`, respectively.
You may provide non-default values to the tolerances and to the iteration budget.

In the succeeding code cell,
replace the `pass` statement with your code.

In [4]:
T0 = 1.0 

print(f"Finding a root starting at {T0}:")
try:
    t_nr, t_nr_info = spo.newton(func=f, x0=T0, fprime=df, full_output=True, maxiter=100)
except Exception as e:
    t_nr = None
    t_nr_info = None
    print("Newton–Raphson failed:", e)

if t_nr is not None:
    print(t_nr_info)
else:
    print("No convergence.")

Finding a root starting at 1.0:
      converged: True
           flag: converged
 function_calls: 10
     iterations: 5
           root: 1.2321436812926325
         method: newton


In [5]:
print(t_nr_info)

      converged: True
           flag: converged
 function_calls: 10
     iterations: 5
           root: 1.2321436812926325
         method: newton


## Results and discussion

Verify if the obtained (approximate) root is indeed the root
by printing the corresponding value of $f\!\left(t\right)$.
The printed display should follow the following format

>> Residual value: 1.23456790e-07

where 1.23456790e-07 is just an exemplar value.

In the succeeding code cell,
replace the `pass` statement with your code.

In [6]:
if t_nr is not None:
    residual = f(t_nr)
    print(f"Residual value: {residual: .8e}")
else:
    print("No valid root to check.")



Residual value: -8.88178420e-16


Print the minimum voltage across the equivalent-inductance element,
such that the output follows the following:

>> The minimum voltage across the equivalent-inductance element is 1.234567 volts.

where 1.234567 is just an examplar value for the said minimum voltage.

In the succeeding code cell,
replace the `pass` statement with your code.

In [7]:
if t_nr is not None:
    v_min = v_L(t_nr)
    print(f"The minimum voltage across the equivalent-inductance element is {v_min: .6f} volts.")
else:
    print("No minimum voltage computed (no valid root).")

The minimum voltage across the equivalent-inductance element is -2.041667 volts.


## Submission instructions

Download this notebook file,
and save with a filename following the pattern
`ACT-05_<section>_<ID number>`,
where the section is as reflected in your Google Classroom.
For example, if your ID number is 2013-0024
and you are enrolled to the M34W12 class,
then your notebook should be named `ACT-05_M34W12_2013-0024.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-10-09*